In [69]:
%reset -sf

In [70]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
# d4 = [(1,0),(0,1),(-1,0),(0,-1)]
# d8 = [(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [71]:
# Python 3 program to count inversions in an array

# Function to Use Inversion Count


def mergeSort(arr):
	n = len(arr)
	# A temp_arr is created to store
	# sorted array in merge function
	temp_arr = [0]*n
	return _mergeSort(arr, temp_arr, 0, n-1)

# This Function will use MergeSort to count inversions


def _mergeSort(arr, temp_arr, left, right):

	# A variable inv_count is used to store
	# inversion counts in each recursive call

	inv_count = 0

	# We will make a recursive call if and only if
	# we have more than one elements

	if left < right:

		# mid is calculated to divide the array into two subarrays
		# Floor division is must in case of python

		mid = (left + right)//2

		# It will calculate inversion
		# counts in the left subarray

		inv_count += _mergeSort(arr, temp_arr,
								left, mid)

		# It will calculate inversion
		# counts in right subarray

		inv_count += _mergeSort(arr, temp_arr,
								mid + 1, right)

		# It will merge two subarrays in
		# a sorted subarray

		inv_count += merge(arr, temp_arr, left, mid, right)
	return inv_count

# This function will merge two subarrays
# in a single sorted subarray


def merge(arr, temp_arr, left, mid, right):
	i = left	 # Starting index of left subarray
	j = mid + 1 # Starting index of right subarray
	k = left	 # Starting index of to be sorted subarray
	inv_count = 0

	# Conditions are checked to make sure that
	# i and j don't exceed their
	# subarray limits.

	while i <= mid and j <= right:

		# There will be no inversion if arr[i] <= arr[j]

		if arr[i] <= arr[j]:
			temp_arr[k] = arr[i]
			k += 1
			i += 1
		else:
			# Inversion will occur.
			temp_arr[k] = arr[j]
			inv_count += (mid-i + 1)
			k += 1
			j += 1

	# Copy the remaining elements of left
	# subarray into temporary array
	while i <= mid:
		temp_arr[k] = arr[i]
		k += 1
		i += 1

	# Copy the remaining elements of right
	# subarray into temporary array
	while j <= right:
		temp_arr[k] = arr[j]
		k += 1
		j += 1

	# Copy the sorted subarray into Original array
	for loop_var in range(left, right + 1):
		arr[loop_var] = temp_arr[loop_var]

	return inv_count


In [91]:
LENGTH = 3

In [98]:
combs = list(itertools.permutations(range(LENGTH), r=3))

In [99]:
val = list(range(LENGTH))
val = [1,2,1]

In [100]:
unseen = set(itertools.permutations(val, r=LENGTH))

for perm in unseen:
    inv = mergeSort(list(perm))
    print(perm, inv)

(1, 2, 1) 1
(2, 1, 1) 2
(1, 1, 2) 0


In [101]:
for _ in range(10000):
    a,b,c = random.sample(combs, 1)[0]
    val[a], val[b], val[c] = val[b], val[c], val[a]
    unseen.discard(tuple(val))

for perm in unseen:
    inv = mergeSort(list(perm))
    print(perm, inv)

In [77]:
srr = """
""".strip()

srr = """aa
""".strip()

with open("/Users/hkmac/Downloads/input.txt") as f:
    srr = f.read().strip()
    
inp = srr.split("\n")